# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [55]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F
#(..)
import numpy as np

In [2]:
from tqdm import tqdm

for i in tqdm(range(10) ):
    print(i)
    

100%|██████████| 10/10 [00:00<00:00, 12066.47it/s]

0
1
2
3
4
5
6
7
8
9


## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [10]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [11]:
"""print out the number of twits"""

# TODO Implement 
print(twits.keys())
print(len(twits["data"]))

dict_keys(['data'])
1548010


### Split Message Body and Sentiment Score

In [12]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [13]:
nltk.download('wordnet')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens   
    Parameters
    ----------
        message : The text message to be preprocessed.
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    
    
#     print("original message", message )
    
    text = message.lower()
    
    # Replace URLs with a space in the message
#     text = 

    text = re.sub(r'http\S+', '', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$[a-zA-Z0-9]*', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@[a-zA-Z0-9]*', ' ', text)
    
    # Replace everything not a letter with a space
    text = re.sub(r'[\W_]+', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokenss = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    
    tokens = list()
    
    for token in tokenss:
        if len(token) > 1:
#             tokens.append( token )
            tokens.append( wnl.lemmatize(token) )

    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
for i in messages[:3]:
    print(preprocess( i ))


['great', 'buy', 'at', '26', '00', 'ill', 'wait']
[]
['staanalystalert', 'for', 'jefferies', 'maintains', 'with', 'rating', 'of', 'hold', 'setting', 'target', 'price', 'at', 'usd', '350', '00', 'our', 'own', 'verdict', 'is', 'buy']


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [15]:
%%time 
#31.7s
tokenized = [preprocess(message) for message in messages]

CPU times: user 2min 59s, sys: 0 ns, total: 2min 59s
Wall time: 2min 59s


In [16]:
print("Number of tokens before removing empty ones: {}".format(len(tokenized)))

Number of tokens before removing empty ones: 1548010


In [17]:
good_tokens = [idx for idx, token in enumerate(tokenized) if len(token) > 0]

In [18]:
print("Number of tokens after removing empty ones: {}".format(len(good_tokens)))

Number of tokens after removing empty ones: 1503359


In [19]:
# %%time
# # TODO Implement
# # tokenized = pass
# tokenized = list()
# for message in messages:
#     tokenized.append(preprocess(message))


In [20]:
tokenized[0]

['great', 'buy', 'at', '26', '00', 'ill', 'wait']

In [21]:
messages[0]

'$FITB great buy at 26.00...ill wait'

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [22]:

from collections import Counter

"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

all_tokens = [word for twit in tokenized for word in twit]
print(all_tokens[:10])
bow = Counter(all_tokens)
print(len(bow))

['great', 'buy', 'at', '26', '00', 'ill', 'wait', 'staanalystalert', 'for', 'jefferies']
121077


In [23]:
# from collections import Counter

# """
# Create a vocabulary by using Bag of words
# """
# # TODO: Implement 
# all_text = ' '.join([' '.join(tokens) for tokens in tokenized])
# counts = Counter(all_text.split())
# bow2 = dict(counts)

# print( len(bow2) )


In [24]:
print(bow)

Counter({'the': 398872, 'to': 379417, 'is': 284881, 'for': 273521, 'on': 241647, 'of': 211343, 'and': 208498, 'in': 205267, 'this': 203557, '39': 198825, 'it': 193524, 'at': 138320, 'will': 128203, 'up': 121524, 'are': 101426, 'you': 94320, 'that': 89672, 'be': 89243, 'short': 86663, '10': 81733, 'what': 79144, 'today': 76239, 'buy': 76082, 'stock': 75454, 'here': 74957, 'just': 73192, 'down': 69345, 'with': 68860, '2018': 68530, 'not': 68305, 'call': 67650, 'day': 66801, 'we': 64755, 'from': 62379, 'all': 59882, 'if': 58999, 'now': 57513, 'out': 56693, 'they': 56356, 'more': 55346, 'report': 55268, 'wa': 55251, 'by': 53964, 'market': 53664, 'but': 53510, 'have': 52959, 'earnings': 52929, 'estimize': 52763, 'go': 52742, 'like': 51916, 'my': 51353, 'analyst': 49821, 'trade': 49529, 'share': 48658, 'going': 48009, 'next': 46969, 'get': 46687, 'new': 46410, 'a': 46184, 'so': 45884, 'see': 44331, 'ha': 44261, 'can': 44221, 'good': 44202, 'week': 43862, 'volume': 43702, 'back': 43048, 'repo

In [25]:
print(type(bow))

<class 'collections.Counter'>


In [26]:
# print("sorted bow dic:", sorted(bow, key=takeSecond) )
# for i in bow.items():
#     print(i,i[0],i[1])
# print( sorted (bow.items(),reverse = True ) ) #keys())

In [27]:
import operator
# print( sorted( bow.items(), key = operator.itemgetter(1), reverse = True) )
print( sorted( bow.items(), key = operator.itemgetter(1), reverse = True)[:5] )

[('the', 398872), ('to', 379417), ('is', 284881), ('for', 273521), ('on', 241647)]


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [29]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 
import operator

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
# freqs = bow #pass

total_count = len(all_tokens) #len(bow)
freqs = {word: count/total_count for word, count in bow.items()}


# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5e-6 #1e-5 #5 

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 10 # pass

# The k most common words in the corpus. Use `high_cutoff` as the k.

sorted_dict_by_value = sorted( bow.items(), key = operator.itemgetter(1), reverse = True)
k = 100
K_most_common = [ word[0] for word in sorted_dict_by_value[:k]  ]
print(K_most_common)

K_most_common2 = [ word[0] for word in bow.most_common(high_cutoff)] 
print(K_most_common2)

# print("the" in K_most_common) #True
# print("a" in K_most_common)   #False

filtered_words = list()

for word in freqs:
    if (freqs[word] > low_cutoff and word not in K_most_common):
        filtered_words.append(word)

# filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
filtered_words2 = list()

# for word in freqs:
#     if (freqs[word] > low_cutoff and word not in K_most_common2):
#         filtered_words2.append(word)

filtered_words2 = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common2)]

print(len(filtered_words) )
print(len(filtered_words2) )

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', '39', 'it', 'at', 'will', 'up', 'are', 'you', 'that', 'be', 'short', '10', 'what', 'today', 'buy', 'stock', 'here', 'just', 'down', 'with', '2018', 'not', 'call', 'day', 'we', 'from', 'all', 'if', 'now', 'out', 'they', 'more', 'report', 'wa', 'by', 'market', 'but', 'have', 'earnings', 'estimize', 'go', 'like', 'my', 'analyst', 'trade', 'share', 'going', 'next', 'get', 'new', 'a', 'so', 'see', 'ha', 'can', 'good', 'week', 'volume', 'back', 'reporting', 'no', 'quot', 'time', 'last', 'or', 'tomorrow', 'price', 'put', 'growth', 'amp', 'still', 'sell', 'after', 'year', 'bear', 'revenue', 'q3', 'eps', 'over', 'long', 'amc', 'look', 'bmo', 'lol', '20', 'some', 'when', '18', 'close', 'than', '50', '08']
['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', '39']
6922
7012


In [30]:
total_num_words = len( all_tokens )
# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {key: value/total_num_words for key, value in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5e-6

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words)

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', '39', 'it', 'at', 'will', 'up', 'are']


7007

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [31]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""
#TODO Implement

'\nSet the following variables:\n    vocab\n    id2vocab\n    filtered\n'

In [32]:
l1 = ["eat","sleep","repeat"] 
  
# creating enumerate objects 
obj1 = enumerate(l1) 
  
print (list(obj1) ) 
print (list(enumerate(l1)))

[(0, 'eat'), (1, 'sleep'), (2, 'repeat')]
[(0, 'eat'), (1, 'sleep'), (2, 'repeat')]


In [33]:
my_list = ['apple', 'banana', 'grapes', 'pear']
for index, value in enumerate(my_list, 1):
    print(index, value)
for index, value in enumerate(my_list, 1):
    print(value, index)

1 apple
2 banana
3 grapes
4 pear
apple 1
banana 2
grapes 3
pear 4


In [34]:
%%time
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
# vocab = bow      # pass

# vocab = {index: word for index, word in enumerate(filtered_words, 1)}
vocab = {word: index for index, word in enumerate(filtered_words, 1)}

# print(vocab)
# {'great': 1, 'buy': 2, 'ill': 3, 'wait': 4, 'staanalystalert': 5, 'jefferies': 6, 'maintains': 7, 'with': 8, 'rating': 9, 'hold': 10, 'setting': 11, 'target': 12, 'price': 13, 'usd': 14, 

CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 2.58 ms


In [35]:
%%time
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab =  dict((v,k) for k,v in vocab.items()) # pass

# print(id2vocab)
# {1: 'great', 2: 'buy', 3: 'ill', 4: 'wait', 5: 'staanalystalert', 6: 'jefferies', 7: 'maintains', 8: 'with', 9: 'rating', 10: 'hold', 11: 'setting', 12: 'target', 13: 'price', 14: 'usd', 15: 'our', 16: 'own', 17: 'verdict', 18: 'heard', 19: 'there', 20: 'guy', 21: 'who', 22: 'know'


CPU times: user 2.46 ms, sys: 0 ns, total: 2.46 ms
Wall time: 2.47 ms


In [36]:
%%time
# tokenized with the words not in `filtered_words` removed.
filtered = []
for twit in tqdm(tokenized):
    filtered.append([word for word in twit if word in filtered_words])

print(filtered)

100%|██████████| 1548010/1548010 [19:18<00:00, 1336.03it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




In [37]:
print(type(filtered),len(filtered) )

<class 'list'> 1548010


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [38]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral



In [40]:
print(n_neutral)

701597


In [41]:
print(N_examples)

1548010


In [42]:
print(keep_prob)

0.3016022730997995


In [43]:
for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [44]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1932724098545037

In [45]:
print(n_neutral)
print(N_examples)
print(keep_prob)

199162
1030473
0.3016022730997995


Finally let's convert our tokens into integer ids which we can pass to the network.

In [46]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [47]:
print(token_ids)
print(sentiments)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [48]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
    
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)

        # dropout layer
        self.dropout = nn.Dropout(dropout)        
        
        # linear and sigmoid layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.lsof = nn.LogSoftmax()

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden       


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # Softmax function
        logps = self.lsof(out)
        
        # reshape to be batch_size first
        batch_size = nn_input.size(1)       
        
        # return last sigmoid output and hidden state
        return logps, hidden    
    

### View Model

In [49]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.5536, -1.6155, -1.9059, -1.4769, -1.5480],
        [-1.5355, -1.6215, -1.9099, -1.4789, -1.5556],
        [-1.5616, -1.6310, -1.8567, -1.4932, -1.5436],
        [-1.5227, -1.6390, -1.9067, -1.4978, -1.5347]])


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [50]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [51]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.7
split_idx = int(len(token_ids)*split_frac)

train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]


In [52]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Training
It's time to train the neural network!

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(7008, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (lsof): LogSoftmax()
)

In [58]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 512
learning_rate = 0.001

clip=5 # gradient clipping


print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1

        hidden = model.init_hidden(labels.shape[0])
        
      
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        model.zero_grad()
        
        output, hidden = model(text_batch, hidden)     
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()       
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()          
        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            val_losses = []
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                #val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.to(device), labels.to(device)
                
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)                

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())     
        
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            

Starting epoch 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/1... Step: 100... Loss: 0.637768... Val Loss: 0.735445
Epoch: 1/1... Step: 200... Loss: 0.670518... Val Loss: 0.733109
Epoch: 1/1... Step: 300... Loss: 0.634234... Val Loss: 0.725811
Epoch: 1/1... Step: 400... Loss: 0.679563... Val Loss: 0.724631
Epoch: 1/1... Step: 500... Loss: 0.548487... Val Loss: 0.725522
Epoch: 1/1... Step: 600... Loss: 0.708707... Val Loss: 0.717378
Epoch: 1/1... Step: 700... Loss: 0.597072... Val Loss: 0.718525
Epoch: 1/1... Step: 800... Loss: 0.631829... Val Loss: 0.719839
Epoch: 1/1... Step: 900... Loss: 0.622960... Val Loss: 0.712533
Epoch: 1/1... Step: 1000... Loss: 0.670480... Val Loss: 0.711675
Epoch: 1/1... Step: 1100... Loss: 0.639171... Val Loss: 0.710854
Epoch: 1/1... Step: 1200... Loss: 0.670098... Val Loss: 0.714114
Epoch: 1/1... Step: 1300... Loss: 0.693415... Val Loss: 0.706288
Epoch: 1/1... Step: 1400... Loss: 0.624202... Val Loss: 0.704312



## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [59]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab]
        
    # Convert words to ids
    tokens = [vocab[token] for token in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1,1) ) )

    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred


In [60]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[ 0.0001,  0.0100,  0.0078,  0.8734,  0.1087]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [61]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [62]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [63]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [64]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


{'symbol': '$AAPL',
 'score': tensor([[ 0.1533,  0.0341,  0.0711,  0.1263,  0.6153]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.